In [6]:
import sqlite3
import csv
import os
from pathlib import Path

def main():
    """Création de la base de données cycliste avec toutes les tables et importation des données CSV."""
    # Créer ou se connecter à la base de données
    db_path = Path("cyclist_database.db")
    
    # Supprimer la base de données si elle existe déjà
    if db_path.exists():
        db_path.unlink()
    
    # Établir la connexion
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Activer les contraintes de clés étrangères
    cursor.execute("PRAGMA foreign_keys = ON;")
    
    # Création de la table User
    cursor.executescript("""
    CREATE TABLE User (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        password TEXT NOT NULL,
        is_staff BOOLEAN DEFAULT 0
    );
    """)
    
    # Création de la table CyclistInfo
    cursor.executescript("""
    CREATE TABLE CyclistInfo (
        user_id INTEGER PRIMARY KEY,
        gender TEXT CHECK(gender IN ('male', 'female')),
        age INTEGER CHECK(age > 0),
        weight REAL CHECK(weight > 0),
        height REAL CHECK(height > 0),
        FOREIGN KEY (user_id) REFERENCES User(id) ON DELETE CASCADE
    );
    """)
    
    # Création de la table Test
    cursor.executescript("""
    CREATE TABLE Test (
        test_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER NOT NULL,
        test_type TEXT CHECK(test_type IN ('protocol_1', 'protocol_2', 'incremental', 'wingate')),
        test_date DATETIME DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (user_id) REFERENCES User(id) ON DELETE CASCADE
    );
    """)
    
    # Création de la table Performances
    cursor.executescript("""
    CREATE TABLE Performances (
        performance_id INTEGER PRIMARY KEY AUTOINCREMENT,
        test_id INTEGER NOT NULL,
        user_id INTEGER NOT NULL,
        time INTEGER NOT NULL,
        oxygen REAL NOT NULL,
        power REAL NOT NULL,
        cadence REAL NOT NULL,
        heart_rate REAL NOT NULL,
        respiration_freq REAL NOT NULL,
        FOREIGN KEY (test_id) REFERENCES Test(test_id) ON DELETE CASCADE,
        FOREIGN KEY (user_id) REFERENCES User(id) ON DELETE CASCADE
    );
    """)
    
    # Insérer quelques utilisateurs et tests d'exemple
    insert_basic_data(cursor)
    
    # Charger et insérer les données du dossier data
    data_folder = Path("datafolder")  # Utilisez le nom correct de votre dossier de données
    if data_folder.exists() and data_folder.is_dir():
        import_csv_data(cursor, conn, data_folder)
    else:
        print(f"Le dossier {data_folder} n'existe pas ou n'est pas un dossier. Aucune donnée CSV n'a été importée.")
    
    # Valider les changements et fermer la connexion
    conn.commit()
    conn.close()
    
    print("Base de données créée avec succès!")

def insert_basic_data(cursor):
    """Insère des données de base dans la base de données."""
    # Insérer des utilisateurs
    cursor.executescript("""
    INSERT INTO User (first_name, last_name, email, password, is_staff) VALUES
        ('Jean', 'Dupont', 'jean.dupont@example.com', 'motdepasse123', 0),
        ('Marie', 'Martin', 'marie.martin@example.com', 'password456', 0),
        ('Admin', 'System', 'admin@cyclistapp.com', 'admin2024!', 1);
    """)
    
    # Insérer des informations de cyclistes
    cursor.executescript("""
    INSERT INTO CyclistInfo (user_id, gender, age, weight, height) VALUES
        (1, 'male', 32, 75.5, 180.0),
        (2, 'female', 28, 62.0, 165.0);
    """)
    
    # Insérer des tests
    cursor.executescript("""
    INSERT INTO Test (user_id, test_type) VALUES
        (1, 'protocol_1'),
        (1, 'wingate'),
        (2, 'incremental');
    """)

def import_csv_data(cursor, conn, data_folder):
    """Importe les données des fichiers CSV du dossier data dans la base de données."""
    # Configuration des fichiers CSV et leurs associations aux tests
    csv_files = {
        "sbj_1_II.csv": {"user_id": 1, "test_id": 1, "test_type": "protocol_1"}
        # Ajoutez d'autres fichiers CSV ici si nécessaire
        # "sbj_2_wingate.csv": {"user_id": 1, "test_id": 2, "test_type": "wingate"},
        # "sbj_3_incremental.csv": {"user_id": 2, "test_id": 3, "test_type": "incremental"}
    }
    
    # Chercher tous les fichiers CSV dans le dossier data
    all_csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]
    print(f"Fichiers CSV trouvés dans le dossier data: {all_csv_files}")
    
    # Parcourir les fichiers CSV configurés
    for csv_file, info in csv_files.items():
        file_path = data_folder / csv_file
        
        if not file_path.exists():
            print(f"Attention: Le fichier {file_path} n'existe pas. Importation ignorée.")
            continue
        
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                csv_reader = csv.DictReader(f)
                
                # Vérification des colonnes requises
                required_cols = ['time', 'Power', 'Oxygen', 'Cadence', 'HR', 'RF']
                if not csv_reader.fieldnames:
                    print(f"Attention: Le fichier {csv_file} est vide ou mal formaté.")
                    continue
                
                missing_cols = [col for col in required_cols if col not in csv_reader.fieldnames]
                
                if missing_cols:
                    print(f"Attention: Colonnes manquantes dans {csv_file}: {missing_cols}")
                    continue
                
                # Préparation de la requête d'insertion
                insert_query = """
                INSERT INTO Performances 
                (test_id, user_id, time, power, oxygen, cadence, heart_rate, respiration_freq)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                """
                
                # Insertion des données
                performance_data = []
                for row in csv_reader:
                    try:
                        performance_data.append((
                            info["test_id"],
                            info["user_id"],
                            int(row['time']),
                            float(row['Power']),
                            float(row['Oxygen']),
                            float(row['Cadence']),
                            float(row['HR']),
                            float(row['RF'])
                        ))
                        
                        # Exécuter par lots pour de meilleures performances
                        if len(performance_data) >= 1000:
                            cursor.executemany(insert_query, performance_data)
                            performance_data = []
                            conn.commit()
                            
                    except (ValueError, KeyError) as e:
                        print(f"Erreur lors du traitement de la ligne {row}: {e}")
                
                # Insérer les données restantes
                if performance_data:
                    cursor.executemany(insert_query, performance_data)
                
                print(f"Données de {csv_file} importées avec succès.")
                
        except Exception as e:
            print(f"Erreur lors de l'importation de {csv_file}: {e}")
    
    # Importer automatiquement les fichiers CSV non configurés
    for csv_file in all_csv_files:
        if csv_file not in csv_files:
            print(f"Fichier CSV {csv_file} trouvé mais non configuré. Pour l'importer, ajoutez-le au dictionnaire 'csv_files'.")

if __name__ == "__main__":
    main()

Fichiers CSV trouvés dans le dossier data: ['sbj_1_II.csv']
Données de sbj_1_II.csv importées avec succès.
Base de données créée avec succès!
